In [6]:
import pandas as pd

In [57]:
prompt_template = """You are an API testing assistant. Your task is to generate test cases based on the given API specification.
Only generate test cases for the specified endpoints. Analyze the provided API parameter description, and extract any example values for the parameter mentioned in the description. 
Then, generate a few additional example values that correspond to, or are in the same category as the provided values.
If there are no example values provided, simply generate values corresponding to the description, and always generate values when possible.

## Handling Double Quotes in Filters
When generating test cases, if the filters parameter contains string values wrapped in double quotes (e.g., STNAME:"West Virginia"), ensure that they are correctly escaped within JSON.
Convert STNAME:"West Virginia" to STNAME:\\\"West Virginia\\\"
This ensures the JSON remains valid while preserving correct formatting for API requests.
Apply this escaping consistently to any other similar cases in filters.

## **Endpoints to Test**
{endpoints_list}

Here is the API Specifications, please read it carefully.
## **API Specification**
{api_spec}
"""

In [59]:
# endpoints we are going to test
endpoints_list = "/check (POST)\n/search (GET)"

# Open API spec
api_spec = """
/check (POST):
  endpoint: /check
  method: POST
  parameters:
  - name: text
    type: string
    required: true
    description: "The text to be checked."
  - name: language
    type: string
    required: true
    enum: ["en-US", "fr-FR", "de-DE"]
"""

# replace the parameter
final_prompt = prompt_template.format(
    endpoints_list=endpoints_list,
    api_spec=api_spec
)

print(final_prompt)

You are an API testing assistant. Your task is to generate test cases based on the given API specification.
Only generate test cases for the specified endpoints. Analyze the provided API parameter description, and extract any example values for the parameter mentioned in the description. 
Then, generate a few additional example values that correspond to, or are in the same category as the provided values.
If there are no example values provided, simply generate values corresponding to the description, and always generate values when possible.

## Handling Double Quotes in Filters
When generating test cases, if the filters parameter contains string values wrapped in double quotes (e.g., STNAME:"West Virginia"), ensure that they are correctly escaped within JSON.
Convert STNAME:"West Virginia" to STNAME:\"West Virginia\"
This ensures the JSON remains valid while preserving correct formatting for API requests.
Apply this escaping consistently to any other similar cases in filters.

## **E

In [61]:
output_spec = """
## **Test Case Output Format Template**
Generate API test cases in **strict JSON format**, without explanations or extra text.  
Each test case must include:
- `endpoint`: The API endpoint
- `method`: HTTP method (`GET`, `POST`, etc.)
- `parameters`: A JSON object containing API parameters
- `expected_response`: The expected HTTP response code (200, 400, etc.)

### **Template Output:**
```json
[
  {
    "endpoint": "/check",
    "method": "POST",
    "parameters": { "para1": "val1", "para2": "val2" },
    "expected_response": 200
  },
  {
    "endpoint": "/check",
    "method": "POST",
    "parameters": { "para1": "", "para2": "val2" },
    "expected_response": 400
  }
]
```
"""

final_prompt += output_spec

In [63]:
print(final_prompt)

You are an API testing assistant. Your task is to generate test cases based on the given API specification.
Only generate test cases for the specified endpoints. Analyze the provided API parameter description, and extract any example values for the parameter mentioned in the description. 
Then, generate a few additional example values that correspond to, or are in the same category as the provided values.
If there are no example values provided, simply generate values corresponding to the description, and always generate values when possible.

## Handling Double Quotes in Filters
When generating test cases, if the filters parameter contains string values wrapped in double quotes (e.g., STNAME:"West Virginia"), ensure that they are correctly escaped within JSON.
Convert STNAME:"West Virginia" to STNAME:\"West Virginia\"
This ensures the JSON remains valid while preserving correct formatting for API requests.
Apply this escaping consistently to any other similar cases in filters.

## **E

## Import endpoints list

In [66]:
for file in file_list:
    file_path = f"./endpoints/{file}.csv"

file_path = "./endpoints/FDIC.csv"
endpoints_df = pd.read_csv(file_path)
endpoints_list = []
for i, r in endpoints_df.iterrows():
    method = r['method']
    path = r['path']
    res = path + " " + f"({method})"
    endpoints_list.append(res)
endpoints_list = "\n".join(endpoints_list)
endpoints_list

'/institutions (GET)\n/locations (GET)\n/history (GET)\n/financials (GET)\n/summary (GET)\n/demographics (GET)'

## Import API specifications YAML

In [69]:
file_path = "./specification/FDIC.yaml"
with open(file_path, 'r', encoding='utf-8') as file:
    yaml_string = file.read()
print(yaml_string)

openapi: 3.0.0
servers:
  - description: API to serve banking industry data to the public.
    url: '/api'
info:
  description: API to serve banking industry data to the public.
  version: 0.1.0
  title: FDIC Bank Data API (Beta)
  contact:
    email: PublicDataFeedback@fdic.gov
tags:
  - name: Structure
    description: Financial institution demographic and location information
  - name: History
    description: List of structure change events
  - name: Financials
    description: Financial Information
  - name: Historical
    description: Historical data from 1934 onward regarding financial institutions.
  - name: Failures
    description: List of bank failures to date
  - name: Summary of Deposits
    description: List of Summary of Deposits
  - name: Demographics
    description: Demographics Information

paths:
  /institutions:
    get:
      tags:
        - Structure
      security:
        - ApiKeyAuth: []
      summary: Get Financial Institutions
      operationId: searchInstit

## Concat to get final prompt

In [72]:
# replace the parameter
final_prompt = prompt_template.format(
    endpoints_list = endpoints_list,
    api_spec = yaml_string
)
final_prompt += output_spec
# print(final_prompt)

## Output the prompt

In [75]:
export_path = "./prompt/FDIC.txt"
with open(export_path, "w", encoding="utf-8") as file:
    file.write(final_prompt)

***

# Combine Process

In [79]:
file_list = ["FDIC", "LargeLanguageTool", "GenomeNexus", "Spotify"]

for file in file_list:
    file_path = f"./endpoints/{file}.csv"

    endpoints_df = pd.read_csv(file_path)
    endpoints_list = []
    for i, r in endpoints_df.iterrows():
        method = r['method']
        path = r['path']
        res = path + " " + f"({method})"
        endpoints_list.append(res)
    endpoints_list = "\n".join(endpoints_list)

    yaml_file_path = f"./specification/{file}.yaml"
    with open(yaml_file_path, 'r', encoding='utf-8') as f:
        yaml_string = f.read()

    # replace the parameter
    final_prompt = prompt_template.format(
        endpoints_list = endpoints_list,
        api_spec = yaml_string
    )
    final_prompt += output_spec

    export_path = f"./prompt/{file}.txt"
    with open(export_path, "w", encoding="utf-8") as f:
        f.write(final_prompt)